In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from tqdm import tqdm

# Load station data
stations_df = pd.read_csv(
    "../../data/trainline/stations.csv",
    sep=";",
    usecols=["name", "latitude", "longitude", "uic", "country"],
).dropna().reset_index().rename(columns={"index": "id"})

# Load city population data
population_df = pd.read_csv("population_cities_europe_latest_coordinates.csv")

# Convert degrees to radians for both datasets
stations_df[["latitude", "longitude"]] = np.radians(stations_df[["latitude", "longitude"]])
population_df[["latitude", "longitude"]] = np.radians(population_df[["latitude", "longitude"]])

# Earth's radius in kilometers
EARTH_RADIUS_KM = 6371  

# Convert latitude/longitude into 3D Cartesian coordinates for KDTree
def latlon_to_cartesian(lat, lon):
    """Convert latitude/longitude in radians to Cartesian coordinates."""
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return np.column_stack((x, y, z))

# Create a KDTree in 3D space
population_tree = cKDTree(latlon_to_cartesian(
    population_df["latitude"].values, population_df["longitude"].values
))

# Function to find the closest city using Haversine distance
def find_closest_city(station):
    """Find the nearest city to a station using KDTree in 3D space."""
    lat, lon = station["latitude"], station["longitude"]
    
    # Convert station lat/lon to Cartesian coordinates
    station_xyz = latlon_to_cartesian(lat, lon)[0]
    
    # Query KDTree
    dist_idx = population_tree.query(station_xyz, k=1)
    
    # Retrieve population and haversine distance
    city_idx = dist_idx[1]
    closest_city = population_df.iloc[city_idx]
    
    # Compute actual Haversine distance
    dlat = closest_city["latitude"] - lat
    dlon = closest_city["longitude"] - lon
    a = np.sin(dlat / 2) ** 2 + np.cos(lat) * np.cos(closest_city["latitude"]) * np.sin(dlon / 2) ** 2
    haversine_dist = 2 * EARTH_RADIUS_KM * np.arcsin(np.sqrt(a))

    return pd.Series([closest_city["population"], city_idx, haversine_dist])

# Apply the function to all stations
tqdm.pandas()
stations_df[["population", "closest_city_id", "distance_km"]] = stations_df.progress_apply(
    find_closest_city, axis=1
)

stations_df

100%|██████████| 22502/22502 [00:07<00:00, 2887.82it/s]


In [7]:
# print dtypes as dict
dtypes_dict = stations_df.dtypes.apply(lambda x: x.name).to_dict()
print(dtypes_dict)

{'id': 'int64', 'name': 'object', 'uic': 'float64', 'latitude': 'float64', 'longitude': 'float64', 'country': 'object', 'population': 'float64', 'closest_city_id': 'float64', 'distance_km': 'float64'}


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

def compute_haversine(lat1, lon1, lat2, lon2):
    """Haversine formula to calculate the distance between two points (in kilometers)."""
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def filter_stations(merged, min_distance_km, max_station_distance_km):
    """Filter merged DataFrame by a minimum computed distance and a maximum station-defined distance."""
    if min_distance_km is not None:
        merged = merged[merged['distance'] >= min_distance_km]
    if max_station_distance_km is not None:
        merged = merged[merged['distance_km'] <= max_station_distance_km]
    return merged

def prepare_merged_df(travel_df, stations_df, starting_station, min_distance_km, max_station_distance_km):
    """
    Prepares a merged DataFrame by:
      - Keeping only the fastest connection for each destination.
      - Merging with station data.
      - Computing the geographical distance from the starting station.
      - Applying distance filters.
    """
    # Keep only the fastest connection per destination station
    travel_df = travel_df.sort_values('duration_seconds').groupby('destination_station_id', as_index=False).first()
    merged = pd.merge(travel_df, stations_df, left_on='destination_station_id', right_on='id', how='left')
    
    # Compute the distance from the starting station to each destination
    lat0, lon0 = starting_station['latitude'], starting_station['longitude']
    merged['distance'] = merged.apply(lambda row: compute_haversine(lat0, lon0, row['latitude'], row['longitude']), axis=1)
    
    # Apply the distance filters
    merged = filter_stations(merged, min_distance_km, max_station_distance_km)
    
    # Avoid division by zero if distance is zero
    merged['distance'] = merged['distance'].replace(0, 0.001)
    # Convert duration from seconds to hours for scoring functions if needed
    merged['duration'] = merged['duration_seconds'] / 3600.0
    return merged

def simple_efficiency_score(merged):
    """
    Simple Efficiency Score:
    Score = sum( population / (duration * (transfers + 1)) )
    """
    merged['score_component'] = merged['population'] / (merged['duration'] * (merged['transfers'] + 1))
    return merged['score_component'].sum()

def distance_adjusted_score(merged):
    """
    Distance-Adjusted Efficiency Score:
    Score = sum( (population * distance) / (duration * (transfers + 1)) )
    """
    merged['score_component'] = (merged['population'] * merged['distance']) / (merged['duration'] * (merged['transfers'] + 1))
    return merged['score_component'].sum()

def exponential_decay_score(merged, lambda_T=0.001, lambda_k=1, lambda_d=0.1):
    """
    Exponential Decay Score:
    Score = sum( population * exp(-lambda_T * duration) * exp(-lambda_k * transfers) * exp(-lambda_d * distance) )
    """
    merged['score_component'] = merged['population'] * \
                                np.exp(-lambda_T * merged['duration']) * \
                                np.exp(-lambda_k * merged['transfers']) * \
                                np.exp(-lambda_d * merged['distance'])
    return merged['score_component'].sum()

def normalized_composite_score(merged, stations_df, alpha=1.0, beta=1.0):
    """
    Normalized Composite Score:
    Score = sum( (normalized_population) / (1 + alpha*(duration/distance) + beta*transfers) )
    where normalized_population = population / max(population)
    """
    max_population = stations_df['population'].max()
    merged['normalized_population'] = merged['population'] / max_population
    merged['score_component'] = merged['normalized_population'] / (
                                  1 + alpha * (merged['duration'] / merged['distance']) + beta * merged['transfers'])
    return merged['score_component'].sum()

def normalized_composite_ratio_score(merged, alpha=1.0, beta=1.0, scaling_factor=1000):
    """
    Normalized Composite Ratio Score:
    For each destination station, compute the ratio of its population to the total population
    of its country. Then use this ratio in the composite score, scaled by a constant factor.
    
    For a destination j in country C:
       R_j = (population_j / total population of C) * scaling_factor
    
    Score = sum( R_j / (1 + alpha*(duration/distance) + beta*transfers) )
    """
    # First, compute total population per country
    country_totals = merged.groupby('country')['population'].transform('sum')
    merged['population_ratio'] = (merged['population'] / country_totals) * scaling_factor
    merged['score_component'] = merged['population_ratio'] / (
                                  1 + alpha * (merged['duration'] / merged['distance']) + beta * merged['transfers'])
    return merged['score_component'].sum()

def normalized_composite_rank_score(merged, stations_df, alpha=1.0, beta=1.0):
    """
    Normalized Composite Rank Score:
    Instead of the actual population, use a rank-based measure. For each destination station,
    compute its population rank within its country and convert that into a normalized rank score.
    
    For a destination station j in country C:
       Let r_j be its rank among stations in C (1 = highest population) and N be the number of stations in C.
       Define R*_j = (N - r_j + 1) / N.
    
    Score = sum( R*_j / (1 + alpha*(duration/distance) + beta*transfers) )
    """
    merged_copy = merged.copy()
    merged_copy['rank'] = merged_copy.groupby('country')['population'].rank(method='min', ascending=False)
    counts = merged_copy.groupby('country')['population'].transform('count')
    merged_copy['rank_norm'] = (counts - merged_copy['rank'] + 1) / counts
    merged_copy['score_component'] = merged_copy['rank_norm'] / (
                                       1 + alpha * (merged_copy['duration'] / merged_copy['distance']) + beta * merged_copy['transfers'])
    return merged_copy['score_component'].sum()

# ---------------------------------------------------------------------
# Parameters for filtering and scoring
MIN_DISTANCE_KM = 5      # Exclude connections with distance less than 5 km
MAX_STATION_DISTANCE_KM = 100  # Ignore stations with a "distance_km" value > 100

# Parameters for composite scores
ALPHA = 1.0
BETA = 1.0

# Dictionary to store scores for each starting station
scores_dict = {}

# Loop through each CSV file in the "travel_times" folder
input_folder = "travel_times_output"
input_files = [file for file in os.listdir(input_folder) if file.startswith('motis_from_') and file.endswith('.csv')]

for file in tqdm(input_files):
    travel_df = pd.read_csv(os.path.join(input_folder, file))
    # Assuming filename format: motis_from_startStationId_timestamp_method.csv
    parts = file.split("_")
    start_station_id = int(parts[2])
    
    # Retrieve the starting station information from stations_df
    starting_station = stations_df.loc[stations_df['id'] == start_station_id].iloc[0]
    if start_station_id in scores_dict:
        raise ValueError(f"Duplicate starting station ID found: {start_station_id}")
    
    # Prepare the merged DataFrame
    merged = prepare_merged_df(travel_df, stations_df, starting_station, MIN_DISTANCE_KM, MAX_STATION_DISTANCE_KM)
    
    # Compute scores (use .copy() to avoid in-place modifications between functions)
    simple_score = simple_efficiency_score(merged.copy())
    distance_score = distance_adjusted_score(merged.copy())
    exponential_score = exponential_decay_score(merged.copy())
    normalized_score = normalized_composite_score(merged.copy(), stations_df, alpha=ALPHA, beta=BETA)
    ratio_score = normalized_composite_ratio_score(merged.copy(), alpha=ALPHA, beta=BETA)
    rank_score = normalized_composite_rank_score(merged.copy(), stations_df, alpha=ALPHA, beta=BETA)
    
    scores_dict[start_station_id] = {
        'simple': simple_score,
        'distance_adjusted': distance_score,
        # 'exponential': exponential_score,
        'normalized': normalized_score,
        'normalized_ratio': ratio_score,
        'normalized_rank': rank_score
    }

# Create a DataFrame from the scores dictionary.
scores_df = pd.DataFrame.from_dict(scores_dict, orient='index')

# Map station id to station name for readability.
scores_df['station_name'] = scores_df.index.map(lambda sid: stations_df.loc[stations_df['id'] == sid, 'name'].values[0])
scores_df = scores_df.set_index('station_name')

# Print rankings for each metric
print("Ranking by Simple Efficiency Score:")
print(scores_df.sort_values('simple', ascending=False)[['simple']])
print("\nRanking by Distance-Adjusted Efficiency Score:")
print(scores_df.sort_values('distance_adjusted', ascending=False)[['distance_adjusted']])
# print("\nRanking by Exponential Decay Score:")
# print(scores_df.sort_values('exponential', ascending=False)[['exponential']])
print("\nRanking by Normalized Composite Score:")
print(scores_df.sort_values('normalized', ascending=False)[['normalized']])
print("\nRanking by Normalized Composite Ratio Score (Country Population Proportion):")
print(scores_df.sort_values('normalized_ratio', ascending=False)[['normalized_ratio']])
print("\nRanking by Normalized Composite Rank Score (Country Population Rank):")
print(scores_df.sort_values('normalized_rank', ascending=False)[['normalized_rank']])


  0%|          | 0/66 [00:00<?, ?it/s]

100%|██████████| 66/66 [00:07<00:00,  8.55it/s]

Ranking by Simple Efficiency Score:
                             simple
station_name                       
Bruxelles-Midi         69413.396551
Paris Gare du Nord     60684.553531
Amsterdam-Centraal     41413.237897
Luxembourg – Gare      32628.762731
Berlin Hbf             29687.210385
Praha hl.n.            22486.372949
Wien Hbf               22280.557419
København              15969.132458
Bratislava hl.st.      14302.895431
Madrid Atocha          13275.749666
Budapest-Keleti        13189.696588
Warszawa-Wschodnia     11420.261293
Stockholm Central       7015.550881
Zagreb                  6451.658138
Roma Termini            5306.098930
Vilnius                 2965.349174
Lisboa Santa Apolónia   1908.087723
Riga                    1709.857827
Helsinki asema           151.603981
București Nord            12.050744
Athens                     0.000000
Ljubljana                  0.000000

Ranking by Distance-Adjusted Efficiency Score:
                       distance_adjusted
station_nam

In [30]:
from IPython.display import display, Markdown

def format_score(value, rank):
    """Format the score with bold and rank if in the top 3."""
    value = int(value) if value > 9 else round(value, 2)
    if value < 0.01:
        value = 0
    if rank == 1:
        return f"\\textbf{{{value}}}(1)"
    elif rank == 2:
        return f"\\textbf{{{value}}}(2)"
    elif rank == 3:
        return f"\\textbf{{{value}}}(3)"
    else:
        return str(value)

# Round scores and determine rankings
latex_df = scores_df.copy()
for column in ['simple', 'distance_adjusted', 'normalized', 'normalized_ratio', 'normalized_rank']:
    # Round values
    latex_df[column] = latex_df[column]
    # Rank values (descending order)
    ranks = latex_df[column].rank(method='min', ascending=False).astype(int)
    # Format scores with bold and rank for top 3
    latex_df[column] = [format_score(value, rank) for value, rank in zip(latex_df[column], ranks)]

# Add a column for the sum of scores
latex_df['sum_of_scores'] = scores_df[['simple', 'distance_adjusted', 'normalized', 'normalized_ratio', 'normalized_rank']].sum(axis=1)

# Sort by the sum of scores
latex_df = latex_df.sort_values('sum_of_scores', ascending=False)

# Replace underscores in station names with spaces for LaTeX compatibility
latex_df.index = latex_df.index.str.replace('_', ' ')

# Generate LaTeX table
latex_table = latex_df.to_latex(escape=False, index=True, columns=[
    'simple', 'distance_adjusted', 'normalized', 'normalized_ratio', 'normalized_rank'
], caption="City Scores by Metric", label="tab:city_scores")

# Save the LaTeX table to a file
with open("city_scores_table.tex", "w") as f:
    f.write(latex_table)

# Display the LaTeX table
display(Markdown(f"```latex\n{latex_table}\n```"))

```latex
\begin{table}
\caption{City Scores by Metric}
\label{tab:city_scores}
\begin{tabular}{llllll}
\toprule
 & simple & distance_adjusted & normalized & normalized_ratio & normalized_rank \\
station_name &  &  &  &  &  \\
\midrule
Bruxelles-Midi & \textbf{69413}(1) & \textbf{666023}(1) & \textbf{0.14}(1) & \textbf{9}(3) & 3.88 \\
Paris Gare du Nord & \textbf{60684}(2) & \textbf{588433}(2) & \textbf{0.12}(2) & \textbf{12}(1) & \textbf{4.47}(1) \\
Amsterdam-Centraal & \textbf{41413}(3) & \textbf{480233}(3) & 0.11 & 9 & 3.34 \\
Berlin Hbf & 29687 & 425822 & 0.12 & \textbf{9}(2) & \textbf{4.01}(3) \\
Wien Hbf & 22280 & 411420 & 0.11 & 8.45 & 3.81 \\
Luxembourg – Gare & 32628 & 381002 & 0.1 & 8.41 & 2.8 \\
Budapest-Keleti & 13189 & 313982 & 0.1 & 7.03 & 3.43 \\
Warszawa-Wschodnia & 11420 & 284589 & \textbf{0.12}(3) & 9 & \textbf{4.04}(2) \\
Praha hl.n. & 22486 & 269545 & 0.09 & 7.75 & 2.87 \\
Madrid Atocha & 13275 & 264138 & 0.09 & 8.47 & 3.46 \\
Bratislava hl.st. & 14302 & 262004 & 0.1 & 7.77 & 3.25 \\
København & 15969 & 228878 & 0.08 & 7.93 & 2.76 \\
Stockholm Central & 7015 & 194842 & 0.08 & 7.63 & 2.85 \\
Zagreb & 6451 & 151474 & 0.07 & 4.91 & 2.16 \\
Roma Termini & 5306 & 108163 & 0.05 & 4.45 & 1.73 \\
Vilnius & 2965 & 86707 & 0.03 & 5.19 & 1.95 \\
Riga & 1709 & 59756 & 0.03 & 2.89 & 1.08 \\
Lisboa Santa Apolónia & 1908 & 48504 & 0.03 & 2.79 & 1.21 \\
Helsinki asema & 151 & 1152 & 0 & 0.53 & 0.02 \\
București Nord & 12 & 111 & 0 & 0.44 & 0 \\
Athens & 0 & 0 & 0 & 0 & 0 \\
Ljubljana & 0 & 0 & 0 & 0 & 0 \\
\bottomrule
\end{tabular}
\end{table}

```